#ATSC 405 Day 9 Assignment
## Bryan Jansens

-------

##Objective: Adapt the rootfind notebook to solve equation (8) from the notes for the saturation vapour pressure $e_{sat}$ and compare with Thompkins (2.13) or (2.15) for 10 temperatures between 0-20$^{\circ}$ C.

The equation to solve is$\\[2mm]$

$$l_v=\left(\phi_v^*-\phi_l\right)T,$$

where $l_v$ is the (temperature-dependent) enthalpy of evaporation, $\phi_v^*$ is the entropy/mass of the saturated vapor phase of water, $\phi_l$ is the entropy/mass of the liquid phase, and $T$ is the temperature. Expressions for $l_v$, $\phi_v^*$, and $\phi_l$ are developed in the notes. Namely, I can replace $l_v$ with expression (13),$\\[2mm]$

$$l_v=\left(c_{pv}-c_l\right)\left(T-T_0\right)+l_0,\\[2mm]$$

where $l_0=2.501\times10^6 \ \mathrm{J \ kg}^{-1}$ is the value of $l_v$ at temperature $T_0=273 \ \mathrm{K}$, $c_{pv}=1870 \ \mathrm{J \ kg}^{-1} \ \mathrm{K}^{-1}$, and $c_{l}=4187 \ \mathrm{J \ kg}^{-1} \ \mathrm{K}^{-1}$. 

I can replace $\phi_l$ with expression (11),$\\[2mm]$

$$\phi_l=c_l\ln\left(\frac{T}{T_p}\right),\\[2mm]$$

where $T_p=273.15 \ \mathrm{K}$ is the temperature at the triple point of water.

Finally, I can replace $\phi_v^*$ with expression (10),$\\[2mm]$

$$\phi_v^*=c_{pv}\ln\left(\frac{T}{T_p}\right)-R_v\ln\left(\frac{e_{sat}}{e_{s0}}\right)+\phi_0,\\[2mm]$$

where $R_v=461.5 \ \mathrm{J \ kg}^{-1} \ \mathrm{K}^{-1}$ is the specific gas constant for water vapour, $e_{s0}=6.11 \ \mathrm{hPa}$, $\phi_0=l_0/T_p$, and $e_{sat}$ is the saturation vapour pressure that I would like to solve for.

Then, as per the notes, I can insert these last three expressions into the first expression and solve for $e_{sat}$ for different values of $T$. The exact expression for $e_{sat}$ is$\\[2mm]$

$$\begin{aligned}
&\frac{l_v}{T}+\phi_l=\phi_v^*\\[2mm]
& \implies \ e_{sat}=e_{s0}\exp\left[-\frac{\frac{l_v}{T}+\phi_l-c_{pv}\ln\left(\frac{T}{T_p}\right)-\phi_0}{R_v}\right].
\end{aligned}$$

In [99]:
#Imports

import numpy as np
import a405thermo.rootfinder as rf
from importlib import reload
reload(rf)

<module 'a405thermo.rootfinder' from 'C:\\Users\\Bryan\\Documents\\ATSC405\\repos\\A405\\a405thermo\\rootfinder.py'>

In [103]:
#Constants

c_pv = 1870.      #J kg^-1 K^-1
c_l = 4187.       #J kg^-1 K^-1
T_0 = 273.        #K
l_0 = 2.501e6     #J kg^-1
T_p = 273.15      #K
R_v = 461.5       #J kg^-1 K^-1
e_s0 = 6.11       #hPa
phi_0 = l_0/T_p

In [54]:
#Get a list of temperatures

temps = [273, 275, 277, 279, 281, 283, 285, 287, 289, 291]

In [126]:
#Define the root-finding function for e_sat

def esat_zero(esat_guess, esat_target, temp):
    """Function we want to find the root of
       input: esat_guess (hPa), esat_target (hPa), temperature (K)
       output: difference between guess and target -- should be zero when x is a root
    """   
    l_v = (c_pv - c_l)*(temp - T_0) + l_0
    phi_l = c_l*np.log(temp/T_p)
    esat_guess = e_s0*np.exp(-((l_v/temp) + phi_l - c_pv*np.log(temp/T_p) - phi_0)/R_v)
    return esat_target - esat_guess

In [127]:
#Define a function that finds the roots using brentq

def e_sat(esat_target, temp):
    """
       input: theta (K), press (Pa)
       output: e_sat (hPa) found by rootfinder
    """     
    #
    #  use theta as guess for bracket and pass theta,press to theta_zero
    #
    brackets = rf.find_interval(esat_zero, esat_target, esat_target, temp)
    print(brackets)
    e_sat_val = rf.fzero(esat_zero, brackets, esat_target, temp)
    return e_sat_val

In [129]:
reload(rf)
temp = 273
#esat_target = 611.2*np.exp((17.67*(temp - 273))/((temp - 273) + 243.5))
esat_target = e_s0*np.exp(((2.5e6)/R_v)*((1/T_0)-(1/temp)))
print(esat_target)
e_sat(esat_target, temp)

6.11


ValueError: Couldn't find a suitable range.